In [24]:
import pandas as pd

# File path (replace with your actual file path if not in the same directory)
file_path = 'U.S._Chronic_Disease_Indicators.csv'

# Read the dataset into a pandas DataFrame
data = pd.read_csv(file_path)

# Display the first few rows to understand the structure
print(data.head())

   YearStart  YearEnd LocationAbbr LocationDesc DataSource     Topic  \
0       2019     2019           AR     Arkansas      BRFSS  Diabetes   
1       2019     2019           ID        Idaho      BRFSS  Diabetes   
2       2019     2019           IN      Indiana      YRBSS     Sleep   
3       2019     2019           IA         Iowa       NVSS    Asthma   
4       2019     2019           IA         Iowa      BRFSS    Asthma   

                                            Question  Response DataValueUnit  \
0                              Diabetes among adults       NaN             %   
1                              Diabetes among adults       NaN             %   
2    Short sleep duration among high school students       NaN             %   
3  Asthma mortality among all people, underlying ...       NaN        Number   
4                        Current asthma among adults       NaN             %   

      DataValueType  ...  TopicID  QuestionID ResponseID DataValueTypeID  \
0  Crude P

In [25]:
# Drop irrelevant columns
columns_to_drop = [
    'Response', 'DataValueFootnoteSymbol', 'DataValueFootnote', 
    'StratificationCategory2', 'Stratification2', 'StratificationCategory3', 
    'Stratification3', 'ResponseID', 'StratificationID2', 
    'StratificationCategoryID3', 'StratificationID3'
]
data_cleaned = data.drop(columns=columns_to_drop)

In [26]:
# Handle missing values
data_cleaned = data_cleaned.dropna(subset=['DataValue'])  # Drop rows where DataValue is missing

# Rename columns for clarity
data_cleaned = data_cleaned.rename(columns={
    'LocationAbbr': 'State',
    'LocationDesc': 'StateName',
    'DataValue': 'Value',
    'YearStart': 'Year'
})


In [27]:
# Filter for relevant topics (e.g., Diabetes)
data_filtered = data_cleaned[data_cleaned['Topic'] == 'Diabetes']

# Check and clean geolocation data
if 'Geolocation' in data_filtered.columns:
    # Extract latitude and longitude from Geolocation column
    data_filtered[['Latitude', 'Longitude']] = data_filtered['Geolocation'].str.extract(r'\((.+), (.+)\)')

    # Extract latitude and longitude from Geolocation column
data[['Longitude', 'Latitude']] = data['Geolocation'].str.extract(r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)')

# Convert the Latitude and Longitude columns to numeric types
data['Latitude'] = pd.to_numeric(data['Latitude'], errors='coerce')
data['Longitude'] = pd.to_numeric(data['Longitude'], errors='coerce')

# Verify the extracted columns
print(data[['Geolocation', 'Latitude', 'Longitude']].head())


C:\Users\info\AppData\Local\Temp\ipykernel_28076\720002467.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered[['Latitude', 'Longitude']] = data_filtered['Geolocation'].str.extract(r'\((.+), (.+)\)')
C:\Users\info\AppData\Local\Temp\ipykernel_28076\720002467.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered[['Latitude', 'Longitude']] = data_filtered['Geolocation'].str.extract(r'\((.+), (.+)\)')


                                     Geolocation   Latitude   Longitude
0   POINT (-92.27449074299966 34.74865012400045)  34.748650  -92.274491
1  POINT (-114.3637300419997 43.682630005000476)  43.682630 -114.363730
2  POINT (-86.14996019399968 39.766910452000445)  39.766910  -86.149960
3   POINT (-93.81649055599968 42.46940091300047)  42.469401  -93.816491
4   POINT (-93.81649055599968 42.46940091300047)  42.469401  -93.816491


In [28]:
# Combine YearStart and YearEnd into a single column
data['Year'] = data['YearStart']  # Use YearStart since they are the same

# Drop the now redundant YearStart and YearEnd columns
data = data.drop(columns=['YearStart', 'YearEnd'])

# Preview the dataset to confirm changes
print(data[['Year']].head())

   Year
0  2019
1  2019
2  2019
3  2019
4  2019


In [29]:
data.to_csv('cleaned_chronic_disease_data_ddw.csv', index=False)
